In [94]:
import os
import gzip
import numpy as np


def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
from sklearn.utils import shuffle

def get_data_with_n_labels_for_each_class(x_train_full, y_train_full, nr_of_labels, num_classes):
    x_train_full, y_train_full = shuffle(x_train_full, y_train_full)

    x_train = []
    y_train = []

    min_queries = nr_of_labels * num_classes
    x_train.extend(x_train_full[0:min_queries])
    y_train.extend(y_train_full[0:min_queries])

    for index in range(min_queries, y_train_full.size):
        x_train.append(x_train_full[index])
        y_train.append(y_train_full[index])
        _, classes_counter = np.unique(np.array(y_train), return_counts=True)
        if np.amin(classes_counter) == nr_of_labels:
            break

    #TODO select random 500 from each class
    return np.array(x_train), np.array(y_train), x_train_full[len(y_train):], y_train_full[len(y_train):]

In [97]:
x_train_full, y_train_full = load_mnist('data', kind='train')
x_test, y_test = load_mnist('data', kind='t10k')
num_classes = 10

x_train, y_train, x_train_remaining, y_train_remaining = get_data_with_n_labels_for_each_class(x_train_full,
                                                                                               y_train_full,
                                                                                               nr_of_labels=1000,
                                                                                               num_classes=num_classes)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [98]:
import keras.layers
from keras.constraints import maxnorm


def make_stem(inputs, filters=[32, 32, 64], shape=(32, 32, 3), kernel_size=(3, 3), padding='same', activation='relu'):
    filter1, filter2, filter3 = filters
    stem = keras.layers.Conv2D(filter1, kernel_size, input_shape=shape, activation=activation,
                               padding=padding)(inputs)
    stem = keras.layers.Conv2D(filter2, kernel_size, input_shape=shape, activation=activation,
                               padding=padding)(stem)
    stem = keras.layers.Conv2D(filter3, kernel_size, input_shape=shape, activation=activation,
                               padding=padding)(stem)
    stem = keras.layers.MaxPooling2D(kernel_size, strides=(2, 2), padding=padding)(stem)
    return stem


def make_skip_connection(input, filter=64, kernel_size=(3, 3), padding='same', activation='relu', dropout=0.2):
    skip = keras.layers.Conv2D(filter, kernel_size, activation=activation, padding=padding)(input)
    layer = keras.layers.Dropout(dropout)(skip)
    layer = keras.layers.Conv2D(filter, kernel_size, padding=padding)(layer)
    merge = keras.layers.add([layer, skip])
    activation = keras.layers.Activation('relu')(merge)
    return activation


def make_main_block(input, filter=64, kernel_size=(3, 3), padding='same', activation='relu', dropout=0.2,
                    pool_size=(2, 2)):
    block = keras.layers.Conv2D(filter, kernel_size, activation=activation, padding=padding)(input)
    block = keras.layers.Dropout(dropout)(block)
    block = keras.layers.Conv2D(filter, kernel_size, activation=activation, padding=padding)(block)
    block = keras.layers.MaxPooling2D(pool_size=pool_size)(block)
    return block


def make_dense_dropout(input, filter, kernel_constraint=maxnorm(3), activation='relu', dropout=0.2):
    dense = keras.layers.Dense(filter, activation=activation, kernel_constraint=kernel_constraint)(input)
    dropout = keras.layers.Dropout(dropout)(dense)
    return dropout

In [99]:
shape = (28, 28, 1)
pool_size = (2, 2)
dropout = 0.2
inputs = keras.Input(shape=shape)
output = make_stem(inputs)

output = keras.layers.MaxPooling2D(pool_size=pool_size)(output)

output = make_main_block(output, 128)
output = make_main_block(output, 256)

output = keras.layers.Flatten()(output)
output = keras.layers.Dropout(dropout)(output)

output = make_dense_dropout(output, 1024)
output = make_dense_dropout(output, 512)

output = keras.layers.Dense(num_classes, activation='softmax')(output)

model = keras.Model(inputs=inputs, outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_22 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 conv2d_23 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 7, 7, 64)         0         
 g2D)                                                      

In [100]:
# from keras.models import Sequential
# from keras.layers import Conv2D, Dense, Flatten

# from keras.models import Sequential
# from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu', name='Convolution-1'))
# model.add(MaxPooling2D(name='MaxPooling2D-1'))
# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', name='Convolution-2'))
# model.add(MaxPooling2D(name='MaxPooling2D-2'))
# model.add(Flatten())
# model.add(Dense(1024, activation='relu', name='Hidden-1'))
# model.add(Dense(512, activation='relu', name='Hidden-2'))
# model.add(Dense(10, activation='softmax', name='Output'))

# model.summary()

In [101]:
def main(x_train, y_train, x_train_remaining, y_train_remaining):
    accuracy = 0
    best_model = None
    model = train_model(epochs=5, batch_size=64, x_train=x_train, y_train=y_train)
    while y_train_remaining.size > 0:
        curr_accuracy = evaluate_model(model)
        if curr_accuracy > accuracy:
            best_model = model
            accuracy = curr_accuracy
            if accuracy > 0.9:
                best_model.save("best.h5")
                break
        x_train, y_train, x_train_remaining, y_train_remaining = classify_high_confidence(x_train_remaining,
                                                                                          y_train_remaining, x_train,
                                                                                          y_train)
        retrain_model(model=model, batch_size=64, epochs=5, x_train=x_train, y_train=y_train)
    curr_accuracy = evaluate_model(model)
    if curr_accuracy > accuracy:
        best_model = model
    best_model.save("best.h5")


def classify_high_confidence(x_train_remaining, y_train_remaining, x_train, y_train):
    input = x_train_remaining.reshape(-1, 28, 28, 1).astype('float32') / 255
    predictions = model.predict(input)
    certainty = predictions.copy()
    certainty = np.max(certainty, axis=1)
    certainty = np.expand_dims(certainty, axis=1)

    label = predictions.copy()
    label = np.argmax(label, axis=1)

    categ_label = to_categorical(label,num_classes = 10)

    certainty_threshold = 0.95
    indices_over_threshold = np.where(np.any(certainty > certainty_threshold, axis=1))

    x_train = np.append(x_train, input[indices_over_threshold], axis=0)
    y_train = np.append(y_train, categ_label[indices_over_threshold], axis=0)
    x_train_remaining = np.delete(x_train_remaining, [indices_over_threshold], axis=0)
    y_train_remaining = np.delete(y_train_remaining, [indices_over_threshold], axis=0)

    print("Remaining unlabeled: " + str(y_train_remaining.size))
    return x_train, y_train, x_train_remaining, y_train_remaining


def evaluate_model(model):
    loss, accuracy = model.evaluate(x_test, y_test)
    print('loss = {}, accuracy = {}'.format(loss, accuracy))
    return accuracy


def train_model(epochs, batch_size, x_train, y_train):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    return model


def retrain_model(epochs, batch_size, model, x_train, y_train):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    return model


In [102]:
# import matplotlib.pyplot as plt
#
# plt.title('Epoch-Accuracy Graph')
# plt.xlabel = 'Epochs'
# plt.ylabel = 'Loss'
# plt.plot(range(1, len(hist.epoch) + 1), hist.history['accuracy'])
# plt.plot(range(1, len(hist.epoch) + 1), hist.history['val_accuracy'])
# plt.legend(['accuracy', 'val_accuracy'])
# plt.show()

In [103]:
main(x_train, y_train, x_train_remaining, y_train_remaining)

Epoch 1/5
131/131 [==============================] - 3s 14ms/step - loss: 1.2194 - accuracy: 0.5247 - val_loss: 0.8133 - val_accuracy: 0.7217
Epoch 2/5
131/131 [==============================] - 1s 11ms/step - loss: 0.6338 - accuracy: 0.7589 - val_loss: 0.5502 - val_accuracy: 0.7893
Epoch 3/5
131/131 [==============================] - 1s 11ms/step - loss: 0.5112 - accuracy: 0.8080 - val_loss: 0.4391 - val_accuracy: 0.8421
Epoch 4/5
131/131 [==============================] - 1s 11ms/step - loss: 0.4420 - accuracy: 0.8348 - val_loss: 0.4279 - val_accuracy: 0.8325
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.4064 - accuracy: 0.8530
loss = 0.40642213821411133, accuracy = 0.8529999852180481
1550/1550 [==============================] - 4s 2ms/step
Remaining unlabeled: 25486
Epoch 1/5
432/432 [==============================] - 6s 12ms/step - loss: 0.1658 - accuracy: 0.9439 - val_loss: 0.0200 - val_accuracy: 0.9952
Epoch 2/5
432/432 [==============================